In [79]:
import pandas as pd
import sklearn as sk
import numpy as np

#载入文件特征文档
features=table=pd.read_csv('item_features.csv')


#提取训练集

data_train=pd.read_excel('train.xlsx')
data_train["衣服ID"]=[i[:12] for i in data_train["衣服ID"]]
usr_train=data_train.groupby("客户ID")

#提取测试集
data_test=pd.read_excel('test.xlsx')
data_test["衣服ID"]=[i[:13] for i in data_test["衣服ID"]]
usr_test=data_test.groupby("客户ID")


#训练集、测试集重新编码
def recoder(od_dataset):
    usr_item_dict={usr:[i for i in groups['衣服ID']]for usr, groups in list(od_dataset)}
    return usr_item_dict

usr_item_dict_train=recoder(usr_train)
usr_item_dict_test=recoder(usr_test)


#最热100商品提取
hot_100_items=data_train["衣服ID"].value_counts()
hot_100_items=hot_100_items[:100].index


#生成训练集
def clf_trainset_gen(history_items,hot_100_items,features):
    
    #生成反馈
    def sample_gen(lst,features):
        frames=[]
        for i in history_items:
            frames.append(features[features['衣服ID'] == i])
        if frames:
            frames=pd.concat(frames)
        return frames
    
    #生成正反馈
    pos=sample_gen(history_items)
    pos['result']=1
    #生成负反馈
    neg=sample_gen(hot_100_items)
    pos['result']=0
    
    sample=pd.concat([pos,neg])
    return sample


#根据用户历史计算Rank
def rank_Compute(usr,real_item)
        
        #检索用户历史
        usr_history=usr_item_dict_train[usr]
        
        #负反馈采样
        hot_100_items=[for i in hot_100_items not in history_items]
        hot_100_items=random.sample(hot_100_items, len(history_items))
        
        
        #调用训练集函数
        clf_transet=clf_trainset_gen(usr_history,hot_100_items,features)
        
        #生成目标变量
        X=clf_transet[:,2:-1]
        Y=clf_transet[:,-1]
        
        
        #训练模型
        from sklearn.naive_bayes import BernoulliNB
        CLF=BernoulliNB()
        CLF.fit(X,Y)

        #预测用户对除正负反馈外所有物品的功用
        dct={}
        for item_else in features["衣服ID"]：
            if item_else not in usr_history and not in hot_100_items:
                XX=features[features["衣服ID"]==item_else][:,2:]
                #模型打分
                marks=CLF.predict_proba([XX])[0]
                dct[item_else]=marks
                
        lst=sorted(dct.items(),key=lambda dct:dct[1])
        lst=[k for (k,v) in lst]
        #计算排名
        Rank=lst.index(real_item)/len(lst)
    
        
        
rank=[]
#对测试集中每一条记录，进行推荐排名
for usr_test in usr_item_dict_test:
    if usr_test in usr_item_dict_train:
        for real_item in usr_item_dict_test:
            rank.append(rank_Compute(usr_test,real_item))
    else:
        #若无法计算则随机推荐
        rank.append(0.5)

rank_score=sum(rank)/len(usr_item_dict_test)  


print(rank_score)
            
        
    


,Unnamed: 0,衣服ID,衣服单价_1,衣服单价_2,衣服单价_3,衣服单价_4,衣服单价_5,衣服单价_6,衣服单价_7,衣服单价_8,...,裙型裤型_X,"裙型裤型_X,喇叭裤",裙型裤型_九分裤,裙型裤型_小脚裤,裙型裤型_小脚裤、紧身裤,裙型裤型_打底裤,裙型裤型_短裤,裙型裤型_紧身牛仔裤,裙型裤型_铅笔小脚裤,裙型裤型_阔腿裤
3,3,JWWC31001100,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
